<a href="https://colab.research.google.com/github/ian-summers/MLB-player-archetypes-by-age/blob/Data-ETL/Manual_ETL_Import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
try:
    import pyspark
except ImportError:
    !pip install pyspark

In [23]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ImportCSV").getOrCreate()

In [24]:

from google.colab import files


uploaded = files.upload()

Saving batting_2023.csv to batting_2023.csv
Saving batting_2022.csv to batting_2022.csv
Saving batting_2021.csv to batting_2021.csv
Saving batting_2020.csv to batting_2020.csv
Saving batting_2019.csv to batting_2019.csv
Saving batting_2018.csv to batting_2018.csv
Saving batting_2017.csv to batting_2017.csv
Saving batting_2016.csv to batting_2016.csv
Saving batting_2015.csv to batting_2015.csv
Saving batting_2014.csv to batting_2014.csv
Saving batting_2013.csv to batting_2013.csv


In [27]:
import pandas as pd
import io

In [28]:
csv_files = [
    "batting_2023.csv",
    "batting_2022.csv",
    "batting_2021.csv",
    "batting_2020.csv",
    "batting_2019.csv",
    "batting_2018.csv",
    "batting_2017.csv",
    "batting_2016.csv",
    "batting_2015.csv",
    "batting_2014.csv",
    "batting_2013.csv",
]

# Read each CSV file and create a DataFrame
dataframes = []
for csv_file in csv_files:
    df = spark.read.csv(csv_file, header=True, inferSchema=True)
    dataframes.append(df)

# Union all DataFrames to create a single master DataFrame
master_df = dataframes[0]
for df in dataframes[1:]:
    master_df = master_df.union(df)

# Show the master DataFrame
master_df.show()

+-----+------+-----------------+---+---+---+---+--------------------+-------------------+----+-----+-----+-----+-----+-----+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----+----+-------+-------------------+-----+-------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+------------------+--------------------+-------------------+------+----+---+
| IDfg|Season|             Name|Age|  G| AB| PA|                 BB%|                 K%|BB/K|  OBP|  SLG|  OPS|  ISO|BABIP|                LD%|                GB%|                FB%|              HR/FB|              Pull%|              Cent%|              Oppo%|              Soft%|               Med%|              Hard%|  EV|  LA|Barrels|            Barrel%|maxEV|HardHit|           HardHit%|             

In [29]:
from pyspark.sql import functions as F

age_count = master_df.select('Age').distinct().count()
print(age_count)

26


In [30]:
age_distribution = master_df.groupBy('Age').count().orderBy('Age')

age_distribution.show(age_distribution.count(), truncate=False)


+---+-----+
|Age|count|
+---+-----+
|19 |3    |
|20 |25   |
|21 |102  |
|22 |243  |
|23 |464  |
|24 |648  |
|25 |767  |
|26 |835  |
|27 |806  |
|28 |714  |
|29 |613  |
|30 |528  |
|31 |449  |
|32 |386  |
|33 |314  |
|34 |237  |
|35 |158  |
|36 |107  |
|37 |72   |
|38 |39   |
|39 |27   |
|40 |11   |
|41 |7    |
|42 |7    |
|43 |3    |
|44 |2    |
+---+-----+



In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit


# Define the age brackets
age_brackets = [(1, 19, 23), (2, 24, 28), (3, 29, 34), (4, 35, 39), (5, 40, 44)]

# Initialize the complete_df DataFrame
complete_df = master_df

# Create a new column 'age_bracket'
age_column = None  # Initialize age_column variable
for bracket in age_brackets:
    bracket_num, start_age, end_age = bracket
# Create a when condition for the current age bracket
    age_condition = (complete_df['Age'] >= start_age) & (complete_df['Age'] <= end_age)
# Update the age_column with the current bracket number
    age_column = when(age_condition, bracket_num).otherwise(age_column)

# Add the 'age_bracket' column to the DataFrame
complete_df = complete_df.withColumn('age_bracket', age_column)

# Show the updated DataFrame
complete_df.show()

+-----+------+-----------------+---+---+---+---+--------------------+-------------------+----+-----+-----+-----+-----+-----+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----+----+-------+-------------------+-----+-------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+------------------+--------------------+-------------------+------+----+---+-----------+
| IDfg|Season|             Name|Age|  G| AB| PA|                 BB%|                 K%|BB/K|  OBP|  SLG|  OPS|  ISO|BABIP|                LD%|                GB%|                FB%|              HR/FB|              Pull%|              Cent%|              Oppo%|              Soft%|               Med%|              Hard%|  EV|  LA|Barrels|            Barrel%|maxEV|HardHit|           HardHit%| 

In [37]:
# # Group by the 'age_bracket' column, compute the count and sort the results by age_bracket
age_bracket_counts_sorted = complete_df.groupBy('age_bracket').count().orderBy('age_bracket')

# Show the count of players in each age bracket sorted by age_bracket
age_bracket_counts_sorted.show()

+-----------+-----+
|age_bracket|count|
+-----------+-----+
|          1|  837|
|          2| 3770|
|          3| 2527|
|          4|  403|
|          5|   30|
+-----------+-----+



In [42]:
# Define the output path including the file name
output_path = "MLB-player-archetypes-by-age.csv"

# Save the DataFrame as CSV
complete_df.write.csv(output_path, header=True, mode="overwrite")

In [40]:
# Define the output path
output_path = "file:///C:/Users/18607/uu/homework/Project-4/baseballdatabank-2023.1.csv"

# Save the DataFrame as CSV
complete_df.write.csv(output_path, header=True, mode="overwrite")